In [11]:
# ideas for a source test
import cobra
import pandas as pd

from refinegems.utility.io import load_model

In [12]:
test_model_path = '/Users/brune/Documents/11_Test_Data/test_SPECIMEN/thesis/Kp_std/03_refinement/step4-smoothing/Kp_std_smooth.xml'
test_model = load_model(test_model_path,'cobra')


-----
### Plot the basic analysis

- one model  -  done more or less
- collection of models

#### Plot for a single report

In [ ]:
from refinegems.utility.io import load_model
from refinegems.classes.reports import ModelInfoReport
import matplotlib.pyplot as plt

test_model_path = '/Users/brune/Documents/11_Test_Data/test_SPECIMEN/thesis/Kp_std/03_refinement/step4-smoothing/Kp_std_smooth.xml'
test_model = load_model(test_model_path,'cobra')

rep = ModelInfoReport(test_model)
fig = rep.visualise()

plt.show()

------
# EGC

In [7]:
import cobra
import argparse
from tqdm import tqdm
from multiprocessing import Pool
from functools import partial
from itertools import product

In [10]:
DISSIPATION_RXNS = {
    "ATP": {"atp_c": -1, "h2o_c": -1, "adp_c": 1, "h_c": 1, "pi_c": 1},
    "CTP": {"ctp_c": -1, "h2o_c": -1, "cdp_c": 1, "h_c": 1, "pi_c": 1},
    "GTP": {"gtp_c": -1, "h2o_c": -1, "gdp_c": 1, "h_c": 1, "pi_c": 1},
    "UTP": {"utp_c": -1, "h2o_c": -1, "udp_c": 1, "h_c": 1, "pi_c": 1},
    "ITP": {"itp_c": -1, "h2o_c": -1, "idp_c": 1, "h_c": 1, "pi_c": 1},
    "NADH": {"nadh_c": -1, "h_c": 1, "nad_c": 1},
    "NADPH": {"nadph_c": -1, "h_c": 1, "nadp_c": 1},
    "FADH2": {"fadh2_c": -1, "h_c": 2, "fad_c": 1},
    "FMNH2": {"fmnh2_c": -1,  "h_c": 2, "fmn_c": 1},
    "Q8H2": {"q8h2_c": -1, "h_c": 2, "q8_c": 1},
    "MQL8": {"mql8_c": -1, "h_c": 2, "mqn8_c": 1},
    "DMMQL8": {"2dmmql8_c": -1, "h_c": 2, "2dmmq8_c": 1},
    "ACCOA": {"accoa_c": -1, "h2o_c": -1, "h_c": 1, "ac_c": 1, "coa_c": 1},
    "GLU": {"glu__L_c": -1, "h2o_c": -1, "akg_c": 1, "nh4_c": 1, "h_c": 2},
    "PROTON": {"h_p": -1, "h_c": 1}
}


RESULTS = {
    "ATP": {"MR": [], "RB": [], "RF": [], "RM": []},
    "CTP": {"MR": [], "RB": [], "RF": [], "RM": []},
    "GTP": {"MR": [], "RB": [], "RF": [], "RM": []},
    "UTP": {"MR": [], "RB": [], "RF": [], "RM": []},
    "ITP": {"MR": [], "RB": [], "RF": [], "RM": []},
    "NADH": {"MR": [], "RB": [], "RF": [], "RM": []},
    "NADPH": {"MR": [], "RB": [], "RF": [], "RM": []},
    "FADH2": {"MR": [], "RB": [], "RF": [], "RM": []},
    "FMNH2": {"MR": [], "RB": [], "RF": [], "RM": []},
    "Q8H2": {"MR": [], "RB": [], "RF": [], "RM": []},
    "MQL8": {"MR": [], "RB": [], "RF": [], "RM": []},
    "DMMQL8": {"MR": [], "RB": [], "RF": [], "RM": []},
    "ACCOA": {"MR": [], "RB": [], "RF": [], "RM": []},
    "GLU": {"MR": [], "RB": [], "RF": [], "RM": []},
    "PROTON": {"MR": [], "RB": [], "RF": [], "RM": []}
}

# maybe unnessesary 
def is_reversible(reaction) -> bool:
    """Checks reversibility of a reaction.

    Args:
        reaction (model.reactions): Reaction from cobra.Model

    Returns:
        bool: True if reaction is reversible. False otherwise.
    """
    if reaction.reversibility:
        return True
    return False


# @TODO: make independant from BiGG/ID
def check_metab_integration(metabolites: dict[str: int], model: cobra.Model) -> bool:
    """Checks if metabolites from a dictionary are completley integrated in cobra.Model

    Args:
        metabolites (dict[str: int]): Dictionary of metabolites with coefficients.
        model (cobra.Model): Model loaded with cobrapy

    Returns:
        bool: True if all metabolites are completley integrated in model
    """
    for meta in metabolites.keys():
        if meta not in model.metabolites:
            return False
    return True

def add_DISSIPATIONRXNS(model: cobra.Model) -> cobra.Model:
    """Adds DISSPATION RXNS to model if fully integrated.

    Args:
        model (cobra.Model): input cobra model

    Returns:
        cobra.Model: outputs model
    """
    for name, metabolites in DISSIPATION_RXNS.items():
        if "DISSI_"+name not in model.reactions:
            if check_metab_integration(metabolites, model):
                rea = cobra.Reaction(name=name, id="DISSI_"+name)
                model.add_reactions([rea])
                rea.add_metabolites(metabolites)

    return model


def limit_bounds(model: cobra.Model) -> cobra.Model:
    """Limits upper and lower bounds of
    exchange reactions to (0, 0)
    reversible reactions to (-1, 1)
    irreversible reactions to (0, 1)

    excludes dissipation reactions

    Args:
        model (cobra.Model): cobrapy model

    Returns:
        cobra.Model: cobrapy model
    """
    external_comp = cobra.medium.find_external_compartment(model)
    # set fluxes for each reaction within model
    for rea in model.reactions:
        # except dissipation reactions
        if "DISSI_" in rea.id:  
            continue
        # turn off exchange reactions
        elif cobra.medium.is_boundary_type(rea,'exchange',external_comp):
            rea.bounds = (0.0, 0.0)
        # limit reversible reactions to [-1, 1] -> flux 0.1
        elif rea.reversibility: 
            rea.bounds = (-1.0, 1.0)
        # limit irreversible reactions to [0, 1] -> flux 0.1
        else: 
            rea.bounds = (0.0, 1.0)

    return model

# naming dangerous, as it returns True when EGC is NOT present
def check_single_egc(model: cobra.Model, egc: str) -> bool:
    """Checks if an EGC is NOT present. E.g. objective value of model optimized
    for DISSI_ATP > 0.

    Args:
        model (cobra.Model): cobra Model
        egc (str): Name of the dissipation reaction, e.g. ATP

    Returns:
        bool: True if objective value == 0.0. False if objective value > 0.0.
    """
    with model:  # need to add dissi reas & limit bounds each time
        rea_id = "DISSI_"+egc
        # add dissipation rxns -> needed to added each time
        model = add_DISSIPATIONRXNS(model)
        # set fluxes for each reaction within model
        mod_model = limit_bounds(model)

        if rea_id in mod_model.reactions:
            mod_model.objective = rea_id
            solution = mod_model.optimize()
            if solution.objective_value > 0.0:
                return False
            else:
                return True

# why rich? 
# ......            
def sim_growth_richMedium(model: cobra.Model, threshold: float = 0.0) -> bool:
    """Simulates growth of a model in rich medium. Sets the model objective to
    'Growth' before simulation. Returns True if growth exceeds threshold.

    Args:
        model (cobra.Model): cobra Model
        threshold (float, optional): Threshold which objective value has to exceed.
        Defaults to 0.0.

    Returns:
        bool: True if obj_val >= threshold. False otherwise.
    """
    with model as mod_model:
        # reset model objective to growth reaction, just in case
        mod_model.objective = "Growth"  # take care -> this might be different in other models
        solution = mod_model.optimize()

        if solution.objective_value > threshold:
            return True

    return False

# ......
def check_egc_growth(model: cobra.Model, egc: str) -> bool:
    """Checks a model if a specified EGC is NOT present & the model can grow in rich medium.

    Args:
        model (cobra.Model): GEM loaded with cobrapy
        egc (str): Name of the EGC

    Returns:
        bool: True == EGC missing & model can grow; False otherwise
    """
    if check_single_egc(model, egc):
        if sim_growth_richMedium(model):
            return True
        else:
            return False
    else:
        return False

def find_egcs(model: cobra.Model) -> tuple[dict, dict]:
    """Checks a cobra.Model for 15 Energy Generating Cycles (EGCs) specified in 
    DISSIPATION_RXN by adding the dissipation reactions turning off exchange reactions
    and limit other reactions to 0.1% flux.

    Based on Fritzemaier et al. (2017, https://doi.org/10.1371/journal.pcbi.1005494)    

    Args:
        model (cobra.Model): GEM read in with cobra

    Returns:
        tuple[dict, dict]: (egc_reactions, objective_values)
    """
    # add 15 energy dissipation reactions
    with model as mod_model:  

        # ensure no model modifications
        mod_model = add_DISSIPATIONRXNS(mod_model)

        # set fluxes for each reaction within model
        mod_model = limit_bounds(mod_model)

        # set each dissipation reaction as objective & optimize
        egc_reactions = {}
        obj_vals = {}
        for name in DISSIPATION_RXNS.keys():
            rea_id = "DISSI_"+name
            if rea_id in mod_model.reactions:
                mod_model.objective = "DISSI_"+name
                solution = mod_model.optimize()
                fluxes = solution.fluxes
                objval = solution.objective_value

                if objval > 0.0:  # optimization > 0 --> EGC detected
                    obj_vals[name] = objval
                    egc_reactions[name] = {}

                    for rea, flux in fluxes.items():
                        # cutoff for flux 1.0e-10 (=no growth) 
                        if flux < 1.0e-10 and not rea.startswith("DISSI"):
                            egc_reactions[name][rea] = flux

    return (egc_reactions, obj_vals)

def test_modifications(reaction: cobra.Reaction, model: cobra.Model, present_egc: dict, results: dict) -> dict:
    """Tries four cases for a Reaction:
    1. if reaction is not reversible -> make reaction reversible (MR)
    2. limit backward reaction (RB)
    3. limit forward reaction (RF)
    4. "delete" reaction by setting fluxes to 0 (RM)
    -> for each case the EGCs which are present in the model are checked if they are removed
    -> if EGCs are removed we check if the model still grows on optimal medium
    => When both limitations are True reaction is saved to corresponding dictionary

    Args:
        reaction (cobra.Reaction): Reaction from a cobra.Model
        model (cobra.Model): The corresponding GEM loaded with cobrapy
        present_egc (dict): Dictionary of present EGCs {"egc": {}} -> EGCs are keys
        results (dict): "Empty" dictionary of dictionary formated of results {"egc": {"MR":[], "RB":[], "RF":[], "RM":[]}}

    Returns:
        dict: {"egc": {"MR":[potential_solutions],
                       "RB":[potential_solutions],
                       "RF":[potential_solutions],
                       "RM":[potential_solutions]}}
    """
    for egc in present_egc:  # skip egc which are not present -> increase performance
        with model:
            if not is_reversible(reaction):  # skip if reaction is already reversible
                reaction.bounds = (-1000.0, 1000.0)  # irreversible -> reversible
                if check_egc_growth(model, egc):
                    results[egc]["MR"].append(reaction.id)  # MR = make reversible

            reaction.bounds = (0, 1000.0)  # limit backward reaction
            if check_egc_growth(model, egc):
                results[egc]["RB"].append(reaction.id)   # Remove Backward (RB) reaction

            reaction.bounds = (-1000.0, 0)  # limit forward reaction
            if check_egc_growth(model, egc):
                results[egc]["RF"].append(reaction.id)   # Remove Forward (RF) reaction

            reaction.bounds = (0, 0)  # "delete" reaction
            if check_egc_growth(model, egc):
                results[egc]["RM"].append(reaction.id)   # ReMove reaction

    return results


# params not optimal and runtime options need rechecking
# RESULTS as a global param ~~~~maybe change that
def find_mods_resolve_egcs(model: cobra.Model, present_egcs: dict) -> dict:
    """Find the modifications to reactions in a cobra.Model and returns these in a dictionary.
    Splits the modification check in multiple processes.
    Run-time around 20 min for 1300 reactions (roughly 0.92s/reaction).

    Args:
        model (cobra.Model): input GEM

    Returns:
        dict: Dictionary of potential modifications to resolve EGCs
              {"egc": {"MR":[potential_solutions],
                       "RB":[potential_solutions],
                       "RF":[potential_solutions],
                       "RM":[potential_solutions]}}
    """
    results = RESULTS
    output_list = []
    limit = 8

    print("_____________________________________")
    print(f"Try to resolve the following EGCs: {[egc for egc in present_egcs]} \nThis might take a while...")

    # might limit processes with Pool(process=limit) -> otherwise it consumes all cores
    # limit to half of machine cores -> at least for me no speed increment with more cores
    with Pool(processes=limit) as pool:
        # partial -> creates function with fixed variables to call with each iteration
        # needed since pool.imap cannot do that
        part_test_mods = partial(test_modifications,
                                 model=model,
                                 present_egc=present_egcs,
                                 results=results)
        # increment in chunksize will reduce computation time -> but progressbar update also...
        for res in tqdm(pool.imap(func=part_test_mods, iterable=model.reactions, chunksize=5),
                        total=len(model.reactions),
                        desc="Resolve EGCs"):
            output_list.append(res)

    # merge output_list to the final results
    for output_dict in output_list:
        for egc, modifications in output_dict.items():
            for mod, reactions in modifications.items():
                results[egc][mod] = list(set(results[egc][mod] + reactions))

    return results


def condense_results(results: dict) -> dict:
    """Condenses the output from find_mods_resolve_egcs().
    For easier handling of the dictionary.

    Args:
        results (dict): result from find_mods_resolve_egcs()

    Returns:
        dict: condensed dictionary without empty entries.
    """
    return {key: {inner_key: inner_value for inner_key, inner_value in value.items() if inner_value} for key, value in results.items() if any(value.values())}


def get_all_solutions(result: dict) -> list:
    """Builds all possible solutions to resolve all EGCs.
    Output is in form of:
    {modification: {reaction: [EGCs]}}

    Args:
        result (dict): output from condense_results

    Returns:
        list: list of dictionaries, each representing a possible solution to resolve all EGCs.
    """
    def unflatten(pairs: tuple) -> dict:
        """Intern function: unflattens the solution produced in here.

        Args:
            pairs (tuple): possible solutions in flatted form

        Returns:
            dict: Output dictionary of get_all_solutions()
        """
        m = {}
        for egc, (mod, react) in pairs:
            try:
                m[mod][react].append(egc)
            except KeyError:
                try:
                    m[mod].update({react: [egc]})
                except KeyError:
                    m[mod] = {react: [egc]}
        return m

    def paths(flat_solution, needle):
        """Intern function to get the paths of a possible solution for an EGC (needle).

        Args:
            flat_solution (_type_): _description_
            needle (_type_): _description_

        Returns:
            _type_: _description_
        """
        return [(mod, react) for (egc, mod, react) in flat_solution if egc == needle]

    def flatten(egcs):
        """Intern function to flatten the input dictionary.

        Args:
            egcs (_type_): _description_

        Returns:
            _type_: _description_
        """
        return [(egc, mod, react) for egc, mods in egcs.items() for mod, reacts in mods.items() for react in reacts]

    flat_result = flatten(result)
    egcs = list(set([egc for egc, mod, react in flat_result]))
    flat_solutions = []

    for prod in product(*[[(egc, modreact) for modreact in paths(flat_result, egc)] for egc in egcs]):
        flat_solutions.append(list(prod))

    return [unflatten(solution) for solution in flat_solutions]

# @TODO: scoring matrix to weigh options differently
def find_minimal_changes(all_solutions: list) -> tuple[list, int]:
    """Calculates the score and returns a list of all scores and the index of the minimal score.

    Args:
        all_solutions (list): All possible solutions in form of a list of dictionaries.

    Returns:
        tuple: tuple of (list, int) list == all scores, int == index of lowest score
    """
    sums = []
    for sol in all_solutions:
        counter = 0
        for mod, reas in sol.items(): # values?
            for rea, egcs in reas.items(): # len???
                counter += 1
        sums.append(counter)
        
    return sums, sums.index(min(sums))

#.....not checked below

def remove_backwards(model: cobra.Model, reaction: cobra.Reaction) -> cobra.Model:
    """Removes the backward reaction of given reaction by changing the reaction bounds to (0.0, 1000.0).

    Args:
        model (cobra.Model): Input model
        reaction (cobra.Reaction): Reaction to change.

    Returns:
        cobra.Model: changed cobra.Model
    """
    reaction.bounds = (0.0, 1000.0)
    return model


def remove_forwards(model: cobra.Model, reaction: cobra.Reaction) -> cobra.Model:
    """Removes the forward reaction of given reaction by changing the reaction bounds to (1000.0, 0.0).

    Args:
        model (cobra.Model): Input model
        reaction (cobra.Reaction): Reaction to change.

    Returns:
        cobra.Model: changed cobra.Model
    """
    reaction.bounds = (1000.0, 0.0)
    return model


def make_reversible(model: cobra.Model, reaction: cobra.Reaction) -> cobra.Model:
    """Makes given reaction reversible by changing the reaction bounds to (1000.0, 1000.0).

    Args:
        model (cobra.Model): Input model
        reaction (cobra.Reaction): Reaction to change.

    Returns:
        cobra.Model: changed cobra.Model
    """
    reaction.bounds = (1000.0, 1000.0)
    return model


def apply_modifications(model: cobra.Model, solution: dict) -> cobra.Model:
    """Apply the modifications to reactions in solution to the model.
    4 modifications are possible:
    "RM" -> removes the reaction
    "RB" -> removes the backwards reaction
    "RF" -> removes the forward reaction
    "MR" -> makes reaction reversible

    Args:
        model (cobra.Model): Input model
        solution (dict): Best solution from calculation before.

    Returns:
        cobra.Model: modified cobra.Model.
    """
    for mod, react_list in solution.items():
        for react in react_list.keys():
            reaction = model.reactions.get_by_id(react)
            if type(reaction) is cobra.Reaction:
                if mod == "RM":
                    reaction.delete()
                    print(f"{reaction.id} is removed.")
                elif mod == "RB":
                    model = remove_backwards(model, reaction)
                    print(f"Backward reaction is removed from {reaction.id}.")
                elif mod == "RF":
                    model = remove_forwards(model, reaction)
                    print(f"Forward reaction is removed from {reaction.id}.")
                elif mod == "MR":
                    model = make_reversible(model, reaction)
                    print(f"{reaction.id} is now reversible.")
                else:
                    print(f"{mod} is no viable modification... Something went wrong.")
    return model


def main(input_path, output_path, change_model):
    model = cobra.io.read_sbml_model(input_path)

    # check for egcs before hand -> is solution necessary? -> increases performance -> less egcs to check
    egc_reactions, obj_vals = find_egcs(model)

    # resolve egcs
    results = find_mods_resolve_egcs(model, present_egcs=egc_reactions)

    # bring results to condensed format
    condensed_results = condense_results(results)

    # get all possible solutions
    all_solutions = get_all_solutions(condensed_results)

    # find minimal changes
    score, best_score_idx = find_minimal_changes(all_solutions)

    # print best result
    best_solution = all_solutions[best_score_idx]
    print(f"One optimal solution is: {best_solution}")
    print(f"With a score of: {score[best_score_idx]}")

    # print other results?
    # this is heuristic...

    # TODO save changes
    if output_path and change_model:
        out_model = model
        out_model = apply_modifications(out_model, best_solution)

        # TODO check whole model again in the end to verfy we havent established anyother egcs
        check_egc_reactions, obj_vals = find_egcs(out_model)
        if not check_egc_reactions:
            cobra.io.write_sbml_model(out_model, output_path)
            print(f"Removed EGCs from model.\nModel is saved to {output_path}.")

    elif change_model and not output_path:
        print("No output path provided. Model will not be changed.")

    elif output_path and not change_model:
        print(f"'--change' flag set to {change_model}. Please set to 'TRUE' to change and save the model.")

    else:
        print("Calculated one optimal solution.")

    return 1


In [12]:
find_egcs(test_model)

({}, {})

In [83]:
from refinegems.classes.medium import Medium

from typing import Literal

In [7]:
subs = ['Water [H2O]', 'ATP [Adenosine triphosphate]','ADP','Hydrogen [H(+)]','Phosphate [PO4(3-)]',
        'CDP','GTP [Guanosine triphosphate]', 'GDP [Guanosine diphosphate]', 'UDP [Uridine 5-diphosphate]',
        'Nicotinamide adenine dinucleotide [NAD]', 'Nicotinamide adenine dinucleotide phosphate [NADP]',
        'Flavin adenine dinucleotide oxidized [FAD]', 'FMN [Flavin Mononucleotide]', 'D-Glucose',
        '2-Oxoglutarate [Oxoglutaric acid]', 'Ammonia'
        ]

dissipation_metas = Medium('dissipation metabolites')

for s in subs:
        dissipation_metas.add_substance_from_db(s)

In [76]:
# needed information

# namespace to use
namespace = 'BiGG'

# compartment to search for metabolites
compartment = 'c'
compartment_2 = 'p'

# metabolites needed
dissipation_metas.substance_table

# @TODO
# NOTE: Ammonia in DB is NH3 - this should be NH4 - change it?
# reactions equations to add / factors etc.
DISSIPATION_RXNS = {
    "ATP": {"ATP [Adenosine triphosphate]": -1, "Water [H2O]": -1, "ADP": 1, "Hydrogen [H(+)]": 1, "Phosphate [PO4(3-)]": 1},
    "CTP": {"ctp_c": -1, "Water [H2O]": -1, "CDP": 1, "Hydrogen [H(+)]": 1, "Phosphate [PO4(3-)]": 1},
    "GTP": {"GTP [Guanosine triphosphate]": -1, "Water [H2O]": -1, "GDP [Guanosine diphosphate]": 1, "Hydrogen [H(+)]": 1, "Phosphate [PO4(3-)]": 1},
    "UTP": {"utp_c": -1, "Water [H2O]": -1, "UDP [Uridine 5-diphosphate]": 1, "Hydrogen [H(+)]": 1, "Phosphate [PO4(3-)]": 1},
    "ITP": {"itp_c": -1, "Water [H2O]": -1, "idp_c": 1, "Hydrogen [H(+)]": 1, "Phosphate [PO4(3-)]": 1},
    "NADH": {"nadh_c": -1, "Hydrogen [H(+)]": 1, "Nicotinamide adenine dinucleotide [NAD]": 1},
    "NADPH": {"nadph_c": -1, "Hydrogen [H(+)]": 1, "Nicotinamide adenine dinucleotide phosphate [NADP]": 1},
    "FADH2": {"fadh2_c": -1, "Hydrogen [H(+)]": 2, "Flavin adenine dinucleotide oxidized [FAD]": 1},
    "FMNH2": {"fmnh2_c": -1,  "Hydrogen [H(+)]": 2, "FMN [FLavin Mononucleotide]": 1},
    "Q8H2": {"q8h2_c": -1, "Hydrogen [H(+)]": 2, "q8_c": 1},
    "MQL8": {"mql8_c": -1, "Hydrogen [H(+)]": 2, "mqn8_c": 1},
    "DMMQL8": {"2dmmql8_c": -1, "Hydrogen [H(+)]": 2, "2dmmq8_c": 1},
    "ACCOA": {"accoa_c": -1, "Water [H2O]": -1, "Hydrogen [H(+)]": 1, "ac_c": 1, "coa_c": 1},
    "GLU": {"D-Glucose": -1, "Water [H2O]": -1, "2-Oxoglutarate [Oxoglutaric acid]": 1, "Ammonia": 1, "Hydrogen [H(+)]": 2},
    "PROTON": {"Hydrogen [H(+)]": -1, "Hydrogen [H(+)] transported": 1}
}


In [81]:
def add_DISSIPATIONRXNS(model: cobra.Model) -> cobra.Model:
    """Adds DISSPATION RXNS to model if fully integrated.

    Args:
        model (cobra.Model): input cobra model

    Returns:
        cobra.Model: outputs model
    """
    def check_metab_integration(metabolites: dict[str: int], model: cobra.Model,
                            metab_info:pd.DataFrame, namespace:Literal['BiGG']='BiGG',compartment:list=['c','p']) -> None|dict:

        found_ids = {}

        c1_metab = [_.id for _ in model.metabolites if _.compartment == compartment[0]]
        c2_metab = [_.id for _ in model.metabolites if _.compartment == compartment[1]]

        for meta in list(metabolites.keys())[0:-1]:
            # get metabolite database annotations
            pos_ids = metab_info.substance_table[metab_info.substance_table['name']==meta][['db_type','db_id']]
            # check namespace availability 
            if not any(namespace in _ for _ in list(pos_ids['db_type'])):
                return None
            # check metabolite availability in model
            id = pos_ids[pos_ids['db_type'].str.contains(namespace)]['db_id']
            for i in id:
                match namespace:
                    # BiGG ID needs the compartment with a '_' as suffix
                    case 'BiGG':
                        if len(metabolites) == 2: # special case proton
                            i_p = i + '_' + compartment[1]
                            if i_p not in c2_metab:
                                return None
                            else:
                                found_ids[i_p] = metabolites['Hydrogen [H(+)] transported']
                            i += '_' + compartment[0]
                        else:
                            i += '_' + compartment[0]
                    # use ID directly
                    case _:
                        if len(metabolites) == 2: # special case proton
                            i_p = i 
                            if i_p not in c2_metab:
                                return None
                            else:
                                found_ids[i_p] = metabolites['Hydrogen [H(+)] transported']

                if i not in c1_metab:
                    return None
                else:
                    found_ids[i] = metabolites[meta]
                    break 

        return found_ids

    # retrieve information about dissipation reaction metabolites
    # metab_info = Medium('dissipation reaction metabolites')
    # metab_info.add_subset('DiReM')
    
    # add dissipations reactions to model
    for name, metabolites in DISSIPATION_RXNS.items():
        if "DISSI_"+name not in model.reactions:
            if check_metab_integration(metabolites, model,
                                       metab_info, namespace,compartment):
                rea = cobra.Reaction(name=name, id="DISSI_"+name)
                model.add_reactions([rea])
                rea.add_metabolites(metabolites)

    return model

In [82]:
for name, metab in DISSIPATION_RXNS.items():
    if check_metab_integration(metab, test_model,dissipation_metas):
        print(name)

ATP
GTP
GLU
PROTON


In [108]:
from refinegems.utility.databases import PATH_TO_DB
import sqlite3
import warnings

def add_subset_to_db(name, desc, subs_dict, database: str= PATH_TO_DB, default_perc:float=1.0):

    
    # build connection to DB
    connection = sqlite3.connect(database)
    cursor = connection.cursor()

    # add new subset to subset table
    res = cursor.execute('SELECT 1 FROM subset WHERE name = ?',(name,))
    if not res.fetchone():
        cursor.execute("INSERT INTO subset VALUES(?,?,?)",(None,name,desc,))
        connection.commit()
        res = cursor.execute("SELECT last_insert_rowid() FROM subset")
        subset_id = res.fetchone()[0]

        # add the substance connections to subset2substance
        for s,p in subs_dict.items():

            # set default per if none given
            if not p:
                p = default_perc
            # get substance ID
            res = cursor.execute('SELECT id FROM substance WHERE name = ?',(s,))
            subs_id = res.fetchone()
            if subs_id:
                subs_id = subs_id[0]
                # enter the entry into subset2substance
                cursor.execute('INSERT INTO subset2substance VALUES(?,?,?)',(subset_id,subs_id,p,))
                connection.commit()
            else:
                mes = f'Substance {s} not in database. Not added. Please check your input.'
                warnings.warn(mes)

    else:
        mes = f'Subset name {name} already in database. Cannot add.\nPlease choose a different name or delete the old one.'
        warnings.warn(mes,category=UserWarning)

    # close connection to database
    connection.close()


In [109]:
add_subset_to_db('test','test',{'D-Glucose':0.45})

/var/folders/xd/zl0vk7sj60xcc0zmc4k4mxxc0000gr/T/ipykernel_69158/1578287600.py:40: UserWarning: Subset name test already in database. Cannot add.
Please choose a different name or delete the old one.
  warnings.warn(mes,category=UserWarning)
